28/Dec/2023

# Cropping

Our original 1 x 1.25 km (GeoTiles said) was cut to 500m x 500m.

But in fact, it is not 1 x 1.25 km, maybe because of the projection.

## Code explain

This code cuts the point cloud into different pieces.

But you can see ours data..

---
          x    y
min 187980.0 311230.0

max 189019.999 312519.999

-----

It's an ugly number.

Therefore, when cropped at 500x500, 9 tiles (output_tile_0~8) will appear.

You can see the figure:
https://drive.google.com/file/d/1be5g9w_FwhZfU-x74logKkkVVpLOB30R/view?usp=drive_link


## Code part
The main thing is to make a mask and remove the things that are not in the mask.

mask = (las.x >= x) & (las.x < x + tile_size) & (las.y >= y) & (las.y < y + tile_size)

(las.x >= x): Check whether the "x coordinate of each point (las.x)" in the point cloud is greater than or equal to the current x value (this is the left boundary of bounding).

(las.x < x + tile_size): Check whether the "x coordinate of each point (las.x)" in the point cloud is less than the sum of the current value x and the specified value tile_size (left boundary +500). That is, after +500, check whether it is inside (beyond 500, out of bounds. This is the right bound)
Make sure the x-coordinate is within the x-value range of the specified tile.

(las.y >= y)
(las.y < y + tile_size)
Same meaning

In [ ]:
import laspy
import numpy as np

def read_las(file_path):
    """
    read files
    """
    return laspy.read(file_path)

def get_bounds(las):
    """
    get bounds
    """
    min_x, max_x = las.header.x_min, las.header.x_max
    min_y, max_y = las.header.y_min, las.header.y_max
    return min_x, max_x, min_y, max_y

def cut_into_tiles(las, tile_size=500):
    """
    Crop the point cloud into tiles of specified size
    """
    min_x, max_x, min_y, max_y = get_bounds(las)

    x_range = np.arange(min_x, max_x, tile_size)
    y_range = np.arange(min_y, max_y, tile_size)


    tiles = []
    for x in x_range:
        for y in y_range:
            mask = (las.x >= x) & (las.x < x + tile_size) & (las.y >= y) & (las.y < y + tile_size)

            if np.any(mask):
                tile = las[mask]
                tiles.append(tile)
    return tiles

def save_tiles(tiles, base_name):
    """
    save
    """
    for i, tile in enumerate(tiles):
        tile.write(f"{base_name}_{i}.laz")



In [ ]:
# how to use
'''
file_path = "69DN2_04.LAZ"
las = read_las(file_path)
tiles = cut_into_tiles(las)
save_tiles(tiles, "output_tile")
'''

# Result

0 187980.0 311230.0 lower left

1 187980.0 311730.0 Center left

2 X 187980.0 312230.0 Upper left

3 188480.0 311230.0 middle botton

4 188480.0 311730.0 center

5  X                middle upper

6X

7X

8X

#### 0 1 3 4 can be used
#### I use 4 
